In [ ]:
import re
def assign_category(description): #EXXO NMOBIL
    if re.search( r"(^EXXO\s*N|^SPEEDWAY|^MAIN\s+ST\s+GAS|^SUNOCO|GULF\s+OIL|CITGO|MAIN ST(REET)? GAS).*?", str(description), re.IGNORECASE):
        return "Gas"
    elif re.search( r"^Costco\s+Annual\s+Membership\s+Renewal", str(description), re.IGNORECASE):
        return "Fees and Adjustments"
    elif re.search( r"(^COST\s*CO\s+WHSE.*?|WWW COSTCO COM)", str(description), re.IGNORECASE):
        return "Costco"
    elif re.search( r"^AMZN\s+.*?\s+GIFT\s+CARD.*?", str(description), re.IGNORECASE):
        return "Gift Card"
    elif re.search(r"(MCDONALD(')*S|TANDOORI|DOMINOS|5GUYS|PANDA PAVILLION|AMERICAN STEAKHOUSE|GYU-KAKU|TERIYAKI ONE|KARAVALLI|JAMROCK JERK|AUNTIE ANNES|BURGER KING|CAFE 1585|CENTRAL SEAFOOD HARTSDALE NY)", str(description),re.IGNORECASE):
        return "Food & Drink"
    elif re.search(r"(KITCHEN PARADISE|STARBUCKS|POPEYES|SGD DUBU|KUMO ULTIMATE SUSHI|Dinosaur Bar-B-Que|THAI SPICE|PANERA|DUNKIN|BIBBLE \& SIP|SHAKE SHACK|BBQ CHICKEN|\#1 ISTANBUL KEBAB HOUSE|TONY AND BENNYS|RED LOBSTER)", str(description),re.IGNORECASE):
        return "Food & Drink"
    elif re.search(r"(ENCHANTED SZECHUAN NORWALK CT|FIVE GUYS|FOOD COURT JAMAICA NY|GYRO GYRO HARTSDALE NY|GYROWORLD FLUSHING NY|HSD FOOD COURT|KFC|Kumo Sush|Lao Si Chua|MID HUDSON BUFFET KINGSTON NY|MINAR|MISTI CAFE|PEPPER LUNCH|POLLO A LA BRASA MIST)", str(description),re.IGNORECASE):
        return "Food & Drink"   
    elif re.search(r"(TACO BELL|TANDORI TASTE OF INDIA|THAI BASIL|THE BUFFET COLLEGE POINT NY|GO! GO! CURRY|WETZEL'S|Yellowtail Hibachi|YUM YUM TOO|ZIBETTO ESPRESSO|BUFFALO WILD WINGS|FANTASY CUISINE)", str(description),re.IGNORECASE):
        return "Food & Drink"
    elif re.search(r"(PATISSERIE DIDIER DUMAS NYACK NY)", str(description),re.IGNORECASE):
        return "Food & Drink"
    elif re.search(r"(TRADER JOE|H MART|FUJI MART|SHOPRITE|WAL-MART|ALDI|ASIA BAZAAR|CHANG JIANG SUPERMARKE FLUSHING NY|GOLD CITY SUPERMARKET FLUSHING NY|GOOD FORTUNE SUPERMARK|HUA LIAN SUPERMARKET|LA PLACITA|C \& A SUPERMARKET|STEW LEONARDS|GMART MILFORD CT)", str(description),re.IGNORECASE):
        return "Groceries"
    elif re.search(r"(KUMO GROCERY STAMFORD CT)", str(description),re.IGNORECASE):
        return "Groceries"
    elif re.search(r"(TMOBILE|CT NATURAL GAS|CONNECTICUT NATURAL)", str(description),re.IGNORECASE):
        return "Bills & Utilities"
    elif re.search(r"(CHATGPT|OPENAI)", str(description),re.IGNORECASE):
        return "Subscriptions"
    elif re.search(r"(CVS\/PHARMACY)", str(description),re.IGNORECASE):
        return "Pharmacy"
    elif re.search(r"(MARCUS MAYUS|GandelmanCardiology|RYE ORAL SURGERY|WESTCHESTER AVE DENTAL|GARRICK F WONG|RYE PERIODONTICS)", str(description),re.IGNORECASE):
        return "Health & Wellness"
    elif re.search(r"(EVA AIR|CATHAYPACAIR)", str(description),re.IGNORECASE):
        return "Travel"
    else:
        return "Shopping"

In [22]:
from pypdf import PdfReader
import re
import os
import glob
import pandas as pd

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        reader = PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text

def extract_activities_from_text(text, year, month):
    pattern = r"^(\d{2}\/\d{2})\s+(.*?)\s+((-?\d{1,3}(,\d{3})*)*\.\d{2})"

    lines = []
    for line in text.split("\n"):
        match = re.match(pattern,line) 
        if match:
            if match.group(1)[:2] == "12" and month == "01":
                this_year = str(int(year) - 1)
            else:
                this_year = year
            # lines.append(f"{match.group(1)}/{this_year},{match.group(2)},{match.group(3).replace(',','')}")
            lines.append((f"{match.group(1)}/{this_year}", match.group(2).strip(), match.group(3).replace(',','')))  # Update the tuple at index i

    return lines


directory_path = "/Users/kingwanlau/Downloads/spending/chase"
pdf_files = glob.glob(os.path.join(directory_path, "*.pdf"))
activities = []
for pdf in pdf_files:
    year = os.path.basename(pdf).split("-")[0][:4]
    month = os.path.basename(pdf).split("-")[0][4:6]
    text_data = extract_text_from_pdf(pdf)
    activities += extract_activities_from_text(text_data, year, month)

chase_old_activities_df = pd.DataFrame(activities, columns=['Transaction Date','Description','Amount'])


In [ ]:
import pandas as pd

chase_old_activities_df["Transaction Date"] = pd.to_datetime(chase_old_activities_df["Transaction Date"], format="%m/%d/%Y")
chase_old_activities_df["Amount"] = chase_old_activities_df["Amount"].astype(float)

# delete rows for Auto Payment
chase_old_activities_df = chase_old_activities_df[~chase_old_activities_df["Description"].str.contains(r"^AUTOMATIC\s+PAYMENT", na=False, regex=True)]
chase_old_activities_df = chase_old_activities_df[~chase_old_activities_df["Description"].str.contains(r"^Payment\s+Thank\s+You", na=False, regex=True)]


chase_old_activities_df.insert(2, "Category", "Shopping")
chase_old_activities_df["Category"] = chase_old_activities_df["Description"].apply(assign_category)

chase_old_activities_df = chase_old_activities_df.sort_values(by="Transaction Date", ascending=False)


# chase_old_activities_df[chase_old_activities_df["Category"]=="Shopping"].head(30)
# chase_old_activities_df = chase_old_activities_df.sort_values(by="Transaction Date", ascending=False)



In [ ]:
# activities_pd.to_csv("output.csv")
# activities_pd.to_csv("old.csv", index=False)
# activities_pd[activities_pd["Transaction Date"]<"2023-09-20"].to_csv("old.csv", index=False)
chase_old_activities_df["Transaction Date"].max()


In [ ]:
import matplotlib.pyplot as plt

monthly_category_spending = (
    chase_old_activities_df.groupby([chase_old_activities_df['Transaction Date'].dt.to_period('Y'), 'Category'])['Amount']
    .sum()
    .unstack(fill_value=0)
    .sort_index()
)

monthly_category_spending.index = monthly_category_spending.index.to_timestamp()

plt.figure(figsize=(14,7))
monthly_category_spending.plot(kind='bar', stacked=True, figsize=(16,8))

plt.title('Monthly Spending By Category', fontsize=16)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Spending ($)", fontsize=12)
plt.xticks(rotation=45, ha="right")
plt.legend(title="Category", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


In [ ]:
# chase_old_activities_df[(chase_old_activities_df["Category"] == "Shopping") & (chase_old_activities_df["Transaction Date"] < "2022/01/01")].head(30)
chase_old_activities_df[(chase_old_activities_df["Transaction Date"] >= "2021/01/01") & (chase_old_activities_df["Transaction Date"] < "2022/01/01")].head(30)

# chase_old_activities_df[chase_old_activities_df["Description"].str.contains("TRIP", case=False)]